# Test for game with observabilities

In [147]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [148]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy

In [149]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(util)

<module 'source.util' from 'C:\\Users\\Mattia\\Desktop\\Mattia\\Git\\onlineSG\\source\\util.py'>

In [150]:
time_horizon = 1000

In [151]:
T = 5 #np.random.randint(2,10)
targets = util.gen_targets(T, low=1, high=10)
distributions = []
for i in range(20):
    distributions.append(tuple(util.gen_distr(T)))
values = tuple((v, v) for v in targets)
print(values)
observabilities = util.gen_observabilities_with_len(T)
g = game.GameWithObservabilities(values, time_horizon)
print(T, targets)
print(observabilities)
g.set_observabilities(observabilities)
attacker = [attackers.ObservingStackelbergAttacker(g, 1, 1)]
profiles = []
profiles.append(attackers.ObservingStackelbergAttacker(g, 1, 1))
for d in distributions:
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
profiles.append(attackers.StackelbergAttacker(g, 1))
defender = [player.Defender(g, 0, 1)]
g.set_players(defender, attacker, profiles)

((9, 9), (5, 5), (2, 2), (1, 1), (8, 8))
5 [9, 5, 2, 1, 8]
{0: 0.541, 1: 0.52, 2: 0.848, 3: 0.507, 4: 0.822}


In [152]:
e = runner.Experiment(g)

In [153]:
e.run_interaction_with_observations()

In [156]:
print(e.game.strategy_history[-1])
print(e.game.history[-1])
print(e.game.observation_history[-1])
print(e.environment.last_exp_loss)
print(e.environment.last_opt_loss)

{0: [0.2, 0.2, 0.2, 0.2, 0.2], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [1]}
[(1, 0)]
7.2
4.585987261146497
